# Задание 1

(**NB.** для запуска примеров кода нужен Python версии не ниже **3.10**, допускается использование других версий, в этом случае нужно самостоятельно избавиться от конструкции `match`).

Есть следующий код для [автоматического дифференцирования](https://en.wikipedia.org/wiki/Automatic_differentiation), в котором используются особенности системы типов языка `Python`: 

In [39]:
from dataclasses import dataclass
from typing import Union, Callable
from numbers import Number
import math

@dataclass
class Dual:
    value: float
    d: float

    def __add__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value + o_value, self.d + o_d)
            case Number():
                return Dual(float(other) + self.value, self.d)

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value * o_value, self.value * o_d + self.d * o_value)
            case Number():
                return Dual(float(other) * self.value, float(other) * self.d)    

    __rmul__ = __mul__  # https://docs.python.org/3/reference/datamodel.html#object.__mul__
    __radd__ = __add__  # https://docs.python.org/3/reference/datamodel.html#object.__radd__

    def __sub__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value - o_value, self.d - o_d)
            case Number():
                return Dual(float(other) - self.value, self.d)
    
    def __rsub__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(o_value - self.value, o_d - self.d)
            case Number():
                return Dual(self.value - float(other), -1 * self.d)
                
    # унарные операции 
    def __neg__(self):
        return Dual(-1 * self.value, -1 * self.d)
    def __pos__(self):
        return Dual(self.value, self.d)

    # деление 
    def __truediv__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value / o_value, (self.d * o_value - o_d * self.value) / o_value ** 2)
            case Number():
                return Dual(self.value / float(other), self.d / float(other))

    def __rtruediv__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(o_value / self.value, (o_d * self.value - self.d * o_value) / self.value ** 2)
            case Number():
                return Dual(float(other) / self.value, -1 * self.d * float(other) / self.value ** 2)
    
    # возведение в степень 
    def __pow__(self, other: Number) -> "Dual": 
        return Dual(self.value ** float(other), self.d * float(other) * self.value ** (float(other) - 1))
        
    def __rpow__(self, other: Number) -> "Dual": 
        return Dual(float(other) ** self.value, self.d * float(other) ** self.value * math.log(float(other)))

def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    return lambda x: func(Dual(x, 1.0)).d 

Поддерживаются две операции - сложение и умножение. Применить можно так:

In [40]:
# Функция, которую будем дифференцировать
def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

f_diff = diff(f)

# значение производной в точке x = 2
f_diff(2)

22.0

## Задание 1.1 (5 баллов)

Какие недостатки вы видите в данной реализации? Реализуйте поддержку (полностью самостоятельно или модифицируя приведенный код):
- [унарных операций](https://docs.python.org/3/reference/datamodel.html#object.__neg__) 
- деления
- возведения в степень

Каким образом можно проверить корректность решения?  Реализуйте достаточный, по вашему мнению, набор тестов.

In [41]:
# ваш код

# унарные операции 
def f_neg(x: float) -> float:
    return -x * x + 2 * x + 2
def f_neg2(x: float) -> float:
    return -x
def f_pos(x: float) -> float:
    return +x * x + 2 * x + 2
def f_pos2(x: float) -> float:
    return +x
if(diff(f_neg)(2) != -2.0): print('не пройден тест 1')
if(diff(f_neg2)(2) != -1.0): print('не пройден тест 2')
if(diff(f_pos)(2) != 6.0): print('не пройден тест 3')
if(diff(f_pos2)(2) != 1.0): print('не пройден тест 4')


# деление 
def f_div1(x: float) -> float:
    return 5 * x / x 
def f_div2(x: float) -> float:
    return 5 / x 
def f_div3(x: float) -> float:
    return (x + x + x * x) / x

if(diff(f_div1)(2) != 0.0): print('не пройден тест 5')
if(diff(f_div2)(2) != -1.25): print('не пройден тест 6')
if(diff(f_div3)(2) != 1.0): print('не пройден тест 7')


# возведение в степень 
def f_pow(x: float) -> float:
    return x ** 5
def f_rpow(x: float) -> float:
    return 2 ** (5 + x)

if(diff(f_pow)(2) != 80.0): print('не пройден тест 8')
if(abs(diff(f_rpow)(2) - 88.7228) > 1e-3): print('не пройден тест 9')


## Задание 1.2 (7 баллов)
Придумайте способ и реализуйте поддержку функций:
- `exp()`
- `cos()`
- `sin()`
- `log()`

Добавьте соответствующие тесты

In [42]:
# ваш код
def exp(value: Union["Dual", Number])-> Union["Dual", Number]:
        match value:
            case Dual(o_value, o_d):
                return Dual(math.exp(o_value), o_d * math.exp(o_value))
            case Number():
                return math.exp(value)

def cos(value: Union["Dual", Number])-> Union["Dual", Number]:
        match value:
            case Dual(o_value, o_d):
                return Dual(math.cos(o_value), -1 * o_d * math.sin(o_value))
            case Number():
                return math.cos(value)

def sin(value: Union["Dual", Number])-> Union["Dual", Number]:
        match value:
            case Dual(o_value, o_d):
                return Dual(math.sin(o_value), o_d * math.cos(o_value))
            case Number():
                return math.sin(value)

def log(value: Union["Dual", Number])-> Union["Dual", Number]:
        match value:
            case Dual(o_value, o_d):
                return Dual(math.log(o_value), o_d / o_value)
            case Number():
                return math.log(value)

def f_exp(x: float) -> float:
    return x * exp(x ** 2)
def f_cos(x: float) -> float:
    return cos(2 * x)
def f_sin(x: float) -> float:
    return sin(1 / x)
def f_log(x: float) -> float:
    return log(x ** 2 + 5)

if(abs(diff(f_exp)(2) - 491.383) > 1e-3): print('не пройден тест 10')
if(abs(diff(f_cos)(2) - 1.5136) > 1e-3): print('не пройден тест 11')
if(abs(diff(f_sin)(2) - -0.219396) > 1e-3): print('не пройден тест 12')
if(abs(diff(f_log)(2) - 4 / 9) > 1e-3): print('не пройден тест 13')


## Задание 1.3 (3 балла)

Воспользуйтесь методами **численного** дифференцирования для "проверки" работы кода на нескольких примерах. Например,  библиотеке `scipy` есть функция `derivative`. Или реализуйте какой-нибудь метод численного дифференцирования самостоятельно (**+5 баллов**)

In [43]:
from scipy.misc import derivative

def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

derivative(f, 2.)

22.0

In [44]:
# ваш код

#метод двухсторонней разности  
def m2d(f, x_0, dx):
    dydx = (f(x_0 + dx) - f(x_0 - dx)) / (2 * dx)
    dx /= 2
    new_dydx = (f(x_0 + dx) - f(x_0 - dx)) / (2 * dx)
    while(abs(new_dydx - dydx) > 1e-7):
        if(dx < 1e-7): break
        dydx = new_dydx
        dx /= 2
        new_dydx = (f(x_0 + dx) - f(x_0 - dx)) / (2 * dx)
    return dydx

def f1(x: float) -> float:
    return 5 * x * x + 2 * x + 2

def f2(x: float) -> float:
    return 5 * cos(x * x + 2) * x + 2 * log(exp(6 * x))

def f3(x: float) -> float:
    return exp(1/ x ** 2) / (sin(10 * x) + 2)
    
for x in range(1, 50):
    if(abs(diff(f1)(x) - m2d(f1, x, 0.2)) > 1e-3): print('ошибка на f1 в точке', x)
    if(abs(diff(f2)(x) - m2d(f2, x, 0.2)) > 1e-3): print('ошибка на f2 в точке', x)
    if(abs(diff(f3)(x) - m2d(f3, x, 0.2)) > 1e-3): print('ошибка на f3 в точке', x)


## Задание 1.4 (10 баллов)

Необходимо разработать систему автоматического тестирования алгоритма дифференцирования в следующем виде:
- реализовать механизм генерации "случайных функций" (например, что-то вроде такого: $f(x) = x + 5 * x - \cos(20 * \log(12 - 20 * x * x )) - 20 * x$ )
- сгенерировать достаточно большое число функций и сравнить результаты символьного и численного дифференцирования в случайных точках 

Генерацию случайных функций можно осуществить, например, двумя путями. 
1. Генерировать функцию в текстовом виде, зачем использовать встроенную функцию [eval](https://docs.python.org/3/library/functions.html#eval)

```python
func = eval("lambda x: 2 * x + 5")
assert func(42) == 89 
```

2. Использовать стандартный модуль [ast](https://docs.python.org/3/library/ast.html), который позволяет во время выполнения программы манипулировать [Абстрактным Синтаксическим Деревом](https://ru.wikipedia.org/wiki/%D0%90%D0%B1%D1%81%D1%82%D1%80%D0%B0%D0%BA%D1%82%D0%BD%D0%BE%D0%B5_%D1%81%D0%B8%D0%BD%D1%82%D0%B0%D0%BA%D1%81%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B5_%D0%B4%D0%B5%D1%80%D0%B5%D0%B2%D0%BE).
Например, выражение 

```python
func = lambda x: 2 * x + 5
```

Можно запрограммировать с помощью кода:

```python

expr = ast.Expression(
    body=ast.Lambda(
        args=ast.arguments(
            args=[
                ast.arg(arg='x')
            ],
            posonlyargs=[],
            kwonlyargs=[],
            kw_defaults=[],
            defaults=[]
        ),
        body=ast.BinOp(
            left=ast.BinOp(
                left=ast.Constant(value=2),
                op=ast.Mult(),
                right=ast.Name(id='x', ctx=ast.Load())
            ),
            op=ast.Add(),
            right=ast.Constant(value=5)
        )
    )
)

ast.fix_missing_locations(expr)

func = eval(compile(expr, filename="", mode="eval"))

assert func(42) == 89
```

При реализации нужно учитывать области допустимых значений функций.

In [45]:
# ваш код

## Задание 1.5 (7 баллов)

Реализуйте поддержку функций нескольких аргументов. Например

```python
def f(x: float, y: float, z: float) -> float:
    return x * y + z - 5 * y  


f_diff = diff(f)

f_diff(10, 10, 10) # = [10, 5, 1]
```

In [48]:
# ваш код

def diff(func: Callable[[float, float, float], float]):
    return lambda x, y, z: [func(Dual(x, 1.0), y, z).d, func(x, Dual(y, 1.0), z).d, func(x, y, Dual(z, 1.0)).d]

def f(x: float, y: float, z: float) -> float:
    return x * y + z - 5.0 * y 


f_diff = diff(f)

f_diff(10, 10, 10)

[10.0, 5.0, 1.0]